In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
import time

#Scikit Learn
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import  MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

#TensorFlow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Normalization


%matplotlib inline

In [2]:

tourism_df = pd.read_csv('../data/intermediate/cleaned_tourism.csv',index_col=0)
tourism_df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [3]:
# drop columns that don't pertain to the customer's attribute or are arbitraty for our model
tourism_df.drop(columns= ['TypeofContact','DurationOfPitch','NumberOfFollowups','PitchSatisfactionScore'], axis=1, inplace=True)

In [5]:
# For us to categorize our customers, we care about when a profile was correctly pitched, not when it wasn't. So we filter for that. 

tourism_df = tourism_df[tourism_df['ProdTaken'] == 1]
tourism_df.drop(columns = ['ProdTaken'], axis=1,inplace=True)

c:\Users\ingal\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
tourism_df

,CustomerID,Age,CityTier,Occupation,Gender,NumberOfPersonVisiting,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,41.0,3,Salaried,Female,3,Deluxe,3.0,Single,1.0,1,1,0.0,Manager,20993.0
2,200002,37.0,1,Free Lancer,Male,3,Basic,3.0,Single,7.0,1,0,0.0,Executive,17090.0
14,200014,28.0,1,Salaried,Male,2,Basic,3.0,Single,6.0,1,0,0.0,Executive,17028.0
21,200021,NaN,3,Salaried,Male,2,Deluxe,5.0,Single,1.0,0,0,0.0,Manager,18407.0
24,200024,34.0,1,Small Business,Male,2,Basic,5.0,Single,3.0,0,1,1.0,Executive,17960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,49.0,3,Small Business,Male,3,Deluxe,4.0,Single,2.0,1,1,1.0,Manager,26576.0
4884,204884,28.0,1,Salaried,Male,4,Basic,3.0,Single,3.0,1,1,2.0,Executive,21212.0
4885,204885,52.0,3,Salaried,Female,4,Standard,4.0,Married,7.0,0,1,3.0,Senior Manager,31820.0
4886,204886,19.0,3,Small Business,Male,3,Basic,3.0,Single,3.0,0,0,2.0,Executive,20289.0
